   ### 1 前馈神经网络进行MNIST数据集分类实战

In [25]:
import tensorflow as tf
import matplotlib.pyplot as plt

#### 输入层 h0 784 =28*28
#### 隐藏层 h1 256 
#### 隐藏层 h2 128
#### 输出层 h3 10

In [26]:
# 初始化参数
w1 = tf.Variable(tf.random.truncated_normal([784,256],stddev = 0.1)) #一个范围内的正态分布
w2 = tf.Variable(tf.random.truncated_normal([256,128],stddev = 0.1))
w3 = tf.Variable(tf.random.truncated_normal([128,10],stddev = 0.1))
                 
b1 = tf.Variable(tf.zeros([256]))
b2 = tf.Variable(tf.zeros([128]))
b3 = tf.Variable(tf.zeros([10]))

In [27]:
w1


<tf.Variable 'Variable:0' shape=(784, 256) dtype=float32, numpy=
array([[-0.0473101 , -0.1023425 , -0.007532  , ...,  0.12226011,
         0.05271397, -0.15120882],
       [ 0.0586805 ,  0.07880794,  0.04557928, ..., -0.19437794,
        -0.11666639,  0.07392412],
       [ 0.1128264 ,  0.08190385, -0.04114777, ..., -0.0739453 ,
         0.04238528,  0.13701752],
       ...,
       [ 0.1123647 ,  0.02596165,  0.12567185, ...,  0.01706166,
        -0.0364383 ,  0.01764593],
       [ 0.06904992, -0.00629739,  0.0677925 , ...,  0.172207  ,
        -0.03655482, -0.03309697],
       [-0.00405733,  0.08376485,  0.01835093, ..., -0.07563334,
         0.0714797 , -0.10661042]], dtype=float32)>

In [28]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data('C:\jupyter_workspace\MNIST_npz\mnist.npz')

In [29]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [30]:
type(x_train)

numpy.ndarray

In [31]:
x_train = tf.convert_to_tensor(x_train,dtype = tf.float32)/255.
y_train = tf.convert_to_tensor(y_train,dtype = tf.int32)

In [32]:
# (60000,784)
x_train = tf.reshape( x_train ,[-1,784]) #-1就是指任意多个
x_train.shape

TensorShape([60000, 784])

In [33]:
type(x_train)

tensorflow.python.framework.ops.EagerTensor

## 完成一次前向计算

In [34]:
x_train.shape,w1.shape,b1.shape

(TensorShape([60000, 784]), TensorShape([784, 256]), TensorShape([256]))

In [35]:
# h1 :net1(z=sum(wx+b))  out1(relu(z))
#[60000,784]@[784*256]+[256]
net1 = x_train@w1 +tf.broadcast_to(b1,[x_train.shape[0],256])
# net1 = x_train@w1 +b1 #实际上自动广播  @即是矩阵乘法
out1 = tf.nn.relu(net1 )

In [36]:
# h2 :net2(z=sum(wx+b))  out2(relu(z))
#[60000,784]@[784*128]+[128]
net2 = out1 @ w2 + b2
out2 = tf.nn.relu(net2)

In [37]:
# h3 :net3(z=sum(wx+b))  out3(relu(z))
#[60000,128]@[784*256]+256
net3 = out2 @ w3 + b3
out3 = tf.nn.softmax(net3)
out3.shape

TensorShape([60000, 10])

In [38]:
out3

<tf.Tensor: shape=(60000, 10), dtype=float32, numpy=
array([[0.06308786, 0.08336931, 0.08083078, ..., 0.32410648, 0.09790603,
        0.03795485],
       [0.05071559, 0.10446056, 0.05598309, ..., 0.34575722, 0.15220098,
        0.0444854 ],
       [0.06714071, 0.11776586, 0.21202196, ..., 0.14405109, 0.03782846,
        0.03597069],
       ...,
       [0.16233772, 0.11622971, 0.07595493, ..., 0.20473418, 0.08152255,
        0.030886  ],
       [0.1409367 , 0.07707907, 0.08774552, ..., 0.21531983, 0.08783928,
        0.05399894],
       [0.1167386 , 0.11088224, 0.0858992 , ..., 0.18310922, 0.09794047,
        0.04193165]], dtype=float32)>

In [39]:
y_train

<tf.Tensor: shape=(60000,), dtype=int32, numpy=array([5, 0, 4, ..., 5, 6, 8])>

In [40]:
y_train = tf.one_hot(y_train,depth=10)
y_train

<tf.Tensor: shape=(60000, 10), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)>

In [41]:
y_train.shape

TensorShape([60000, 10])

In [42]:
loss = tf.nn.softmax_cross_entropy_with_logits(labels = y_train,logits = out3)
loss = tf.reduce_mean(loss)#求均值
loss

<tf.Tensor: shape=(), dtype=float32, numpy=2.2964115>

### 一次反向传播

In [43]:
with tf.GradientTape() as tape:
    tape.watch([w1,b1,w2,b2,w3,b3])
    out3 = tf.nn.softmax(tf.nn.relu(tf.nn.relu(x_train@w1 +b1 ) @ w2 + b2) @ w3 + b3)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_train,logits = out3))

In [44]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=2.2964115>

In [45]:
grads = tape.gradient(loss,[w1,b1,w2,b2,w3,b3])

In [46]:
lr = 0.01

In [47]:
#更新参数
# w = w-lr*grads


In [ ]:
All_loss=[]
for step in range(1001):
    with tf.GradientTape() as tape:
        tape.watch([w1,b1,w2,b2,w3,b3])
        out3 = tf.nn.softmax(tf.nn.relu(tf.nn.relu(x_train@w1 +b1 ) @ w2 + b2) @ w3 + b3)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_train,logits = out3))
    All_loss.append(loss)
    grads = tape.gradient(loss,[w1,b1,w2,b2,w3,b3])
    #更新参数
    w1 = w1-lr*grads[0]
    b1 = b1-lr*grads[1]
    w2 = w2-lr*grads[2]
    b2 = b2-lr*grads[3]
    w3 = w3-lr*grads[4]
    b3 = b3-lr*grads[5]
    if step%100 ==0:
        print(step,'loss:',float(loss))
    
    

0 loss: 2.2964115142822266
100 loss: 2.2769994735717773
200 loss: 2.248997211456299
300 loss: 2.2043306827545166
400 loss: 2.156979560852051
500 loss: 2.1069881916046143
600 loss: 2.0567824840545654
700 loss: 2.01214599609375
800 loss: 1.972558617591858


In [ ]:
%time plt.plot(All_loss)

## 测试模型

In [ ]:
x_test = tf.convert_to_tensor(x_test,dtype = tf.float32)/255.
y_test = tf.convert_to_tensor(y_test,dtype = tf.int32)
x_test = tf.reshape(x_test ,[-1,784]) #-1就是指任意多个

In [ ]:
out3 = tf.nn.softmax(tf.nn.relu(tf.nn.relu(x_test@w1 +b1 ) @ w2 + b2) @ w3 + b3)
out3

In [ ]:
y_predict = tf.math.argmax(out3,axis=-1)

In [ ]:
y_predict,y_test

In [ ]:
y_test = tf.cast(y_test,tf.int64) #强制转换

In [ ]:
y_c=tf.math.equal(y_predict,y_test)
y_c

In [ ]:
y_c = tf.cast(y_c,tf.int64)
y_c

In [ ]:
r = tf.math.reduce_sum(y_c) / 10000
r.numpy()